In [14]:
# ! pip install PyPDF2
# ! pip uninstall fitz
# ! pip install pymupdf
# ! pip install tabula
# ! pip install pdfplumber


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


ERROR: Could not find a version that satisfies the requirement some_library (from versions: none)
ERROR: No matching distribution found for some_library


In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_vector import MultiVectorRetriever

import pdfplumber
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Function to convert table to text
def table_to_text(table):
    return "\n".join(["\t".join(map(str, row)) for row in table])

# Function to convert HTML table to plain text
def html_to_plain_text(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text(separator="\n")

# Function to convert text to HTML table
def text_to_html_table(table_text):
    rows = table_text.split("\n")
    soup = BeautifulSoup("<table></table>", "html.parser")
    table_tag = soup.table
    for row in rows:
        row_tag = soup.new_tag("tr")
        table_tag.append(row_tag)
        for cell in row.split("\t"):
            cell_tag = soup.new_tag("td")
            cell_tag.string = cell
            row_tag.append(cell_tag)
    return str(table_tag)

# Load and split text from PDF
def load_and_split_pdf_text(pdf_path):
    loader = PyPDFLoader(pdf_path)
    text = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=0)
    return text_splitter.split_documents(text)

# Extract tables from PDF
def extract_tables_from_pdf(pdf_path):
    with pdfplumber.open(pdf_path) as pdf:
        tables = []
        for page in pdf.pages:
            tables.extend(page.extract_tables())
    all_tables_text = [table_to_text(table) for table in tables if table]
    return [text_to_html_table(table_text) for table_text in all_tables_text]

# Load text from both PDFs
pdf1_text_splits = load_and_split_pdf_text('AUTOSAR_CP_SWS_GPTDriver.pdf')  # Generic AUTOSAR document
pdf2_text_splits = load_and_split_pdf_text('path_of_the_pdf')

# Extract tables from both PDFs
pdf1_html_tables = extract_tables_from_pdf('AUTOSAR_CP_SWS_GPTDriver.pdf')  # # Generic AUTOSAR document
pdf2_html_tables = extract_tables_from_pdf('path_of_the_pdf')

# Combine text splits and tables
all_text_splits = pdf1_text_splits + pdf2_text_splits
all_html_tables = pdf1_html_tables + pdf2_html_tables

# Embed text
model = SentenceTransformer('all-MiniLM-L6-v2')
text_embeddings = model.encode([split.page_content for split in all_text_splits])
table_plain_texts = [html_to_plain_text(html_table) for html_table in all_html_tables]
table_embeddings = model.encode(table_plain_texts)

# Store text embeddings in FAISS
dimension = text_embeddings.shape[1]
text_index = faiss.IndexFlatL2(dimension)
text_index.add(np.array(text_embeddings))

# Store table embeddings in FAISS
dimension = table_embeddings.shape[1]
table_index = faiss.IndexFlatL2(dimension)
table_index.add(np.array(table_embeddings))

# Implement a multi-vector retriever
class MultiVectorRetriever:
    def __init__(self, text_index, table_index, model):
        self.text_index = text_index
        self.table_index = table_index
        self.model = model
    
    def retrieve(self, query, k=1):
        query_embedding = self.model.encode([query])
        text_distances, text_indices = self.text_index.search(query_embedding, k)
        table_distances, table_indices = self.table_index.search(query_embedding, k)
        
        retrieved_texts = [all_text_splits[idx].page_content for idx in text_indices[0]]
        retrieved_tables = [all_html_tables[idx] for idx in table_indices[0]]
        
        return retrieved_texts, retrieved_tables

retriever = MultiVectorRetriever(text_index, table_index, model)

# Query the retriever
query = "Gpt_StartTimer?"
retrieved_texts, retrieved_tables = retriever.retrieve(query)

for i, text in enumerate(retrieved_texts):
    print(f"Text {i+1}:\n{text}\n")

for i, table in enumerate(retrieved_tables):
    print(f"Table {i+1}:\n{table}\n")


c:\Users\HTW1KOR\Documents\hackathon\.conda\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\HTW1KOR\Documents\hackathon\.conda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Text 1:
Specification of GPT Driver
AUTOSAR CP R23-11
The function Gpt_StartTimer shall raise the error GPT_E_PARAM_VALUE if the pa-
rameter Value is "0" or not within the allowed range (exceeding the maximum timer
resolution).⌋(SRS_BSW_00323)
[SWS_Gpt_00224] ⌈If development error detection is enabled for GPT module:
If the driver is not initialized, the function Gpt_StartTimer shall raise the error GPT_-
E_UNINIT .⌋(SRS_BSW_00406)
[SWS_Gpt_00084] ⌈If the function Gpt_StartTimer is called on a Channel in state
"running", the function shall raise the runtime error GPT_E_BUSY .⌋()
8.3.7 Gpt_StopTimer
[SWS_Gpt_00285] Definition of API function Gpt_StopTimer ⌈
Service Name Gpt_StopTimer
Syntax void Gpt_StopTimer (
Gpt_ChannelType Channel
)
Service ID [hex] 0x06
Sync/Async Synchronous
Reentrancy Reentrant (but not for the same timer channel)
Parameters (in) Channel Numeric identifier of the GPT channel.
Parameters (inout) None
Parameters (out) None
Return value None
Description Stops a time

In [1]:
! ollama -v

ollama version is 0.1.42


In [2]:
! ollama list

NAME            	ID          	SIZE  	MODIFIED   
codellama:latest	8fdf8f752f6e	3.8 GB	2 days ago	
codegemma:latest	0c96700aaada	5.0 GB	2 days ago	


In [5]:
import subprocess
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Define the function to handle retriever queries (dummy function for now)
def retrieve_query(query):
    # Dummy function to simulate retrieval
    if "GPT" in query:
        return {"retrieved_texts": ["Dummy retrieval for " + query], "retrieved_tables": []}
    else:
        # Implement retrieval logic for other queries
        pass

# Define the few-shot learning examples
few_shot_examples = [
    {
        "query": "Generate C and H code for GPIO initialization on STM32F4",
        "desired_output": {
            "c_code": """
                #include "stm32f4_gpio.h"
                
                void GPIO_Init(GPIO_TypeDef* GPIOx, GPIO_InitTypeDef* GPIO_InitStruct) {
                    // Implementation for GPIO initialization
                }
                """,
            "h_code": """
                #ifndef STM32F4_GPIO_H
                #define STM32F4_GPIO_H
                
                // Header file content for GPIO initialization
                
                #endif // STM32F4_GPIO_H
                """
        }
    },
    {
        "query": "Generate C code for GPT driver",
        "desired_output": {
            "c_code": """
                // C code for GPT driver implementation
                // Include necessary headers and define functions
                """,
            "h_code": """
                // Header file for GPT driver
                // Define data structures and function prototypes
                """
        }
    },
    {
        "query": "Generate C and H code for PORT driver",
        "desired_output": {
            "c_code": """
                // C code for PORT driver implementation
                // Include necessary headers and define functions
                """,
            "h_code": """
                // Header file for PORT driver
                // Define data structures and function prototypes
                """
        }
    },
    {
        "query": "Generate C and H code for SPI driver",
        "desired_output": {
            "c_code": """
                // C code for SPI driver implementation
                // Include necessary headers and define functions
                """,
            "h_code": """
                // Header file for SPI driver
                // Define data structures and function prototypes
                """
        }
    },
    {
        "query": "Generate C and H code for CAN driver",
        "desired_output": {
            "c_code": """
                // C code for CAN driver implementation
                // Include necessary headers and define functions
                """,
            "h_code": """
                // Header file for CAN driver
                // Define data structures and function prototypes
                """
        }
    }
]

# Define a function to retrieve desired output based on query
def retrieve_desired_output(query):
    for example in few_shot_examples:
        if example["query"].lower() == query.lower():
            # Retrieve desired output
            desired_output = example["desired_output"]
            
            # Filter out non-header lines from the .h code
            h_code_lines = desired_output["h_code"].split('\n')
            filtered_h_code_lines = [line for line in h_code_lines if line.strip().startswith('#') or line.strip().startswith('typedef') or line.strip().startswith('struct')]
            desired_output["h_code"] = '\n'.join(filtered_h_code_lines)
            
            return desired_output
    return None

# Start ollama as a background process with interactive mode enabled
command = "nohup ollama serve --interactive &"
process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print("Process ID:", process.pid)

# Prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Local LLM
ollama_llm = "codellama"
model_local = ChatOllama(model=ollama_llm)

# Define the processing chain
chain = (
    {"context": RunnableLambda(retrieve_query), "question": RunnablePassthrough()}
    | prompt
    | model_local
    | StrOutputParser()
)

# Function to generate C and H code based on user query
def generate_mcal_code(query):
    retrieved_output = chain.invoke(query)
    return retrieved_output

# Function to save generated code to a single file
def save_code_to_file(query, file_path):
    generated_code = generate_mcal_code(query)
    with open(file_path, "w") as file:
        file.write("Generated C and H Code for " + query + ":\n")
        file.write(generated_code)

# Test the code generation and save to file for the SPI driver
query = "Generate C and H code for CAN driver"
file_path = "CAN_driver_code.txt"
save_code_to_file(query, file_path)

print("Generated code saved to", file_path)


Process ID: 10028
Generated code saved to CAN_driver_code.txt
